
# Related keywords detection

**NOTE**: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](../ch04/1.setting-up-the-retrotech-dataset.ipynb) notebook.

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("aips-ch6").getOrCreate()

### Step 1: Prepare the data using py-spark and data frames 


## Listing 6.4

In [2]:
#Calculation:
signals_collection="signals"
signals_opts={"zkhost": "aips-zk", "collection": signals_collection}
df = spark.read.format("solr").options(**signals_opts).load()
df.createOrReplaceTempView("signals")
spark.sql("""select lower(searches.target) as keyword, searches.user as user
from signals as searches 
where searches.type='query'
""").createOrReplaceTempView('user_searches')

In [3]:
#Show Results:
spark.sql("""select count(*) rows from user_searches """).show(1)
spark.sql("""select * from user_searches """).show(3)

+------+
|  rows|
+------+
|725459|
+------+

+-------------+-------+
|      keyword|   user|
+-------------+-------+
|power adapter|u304570|
|   jump drive| u26003|
| thor blu ray|u707571|
+-------------+-------+
only showing top 3 rows



### Step2 : Create Cooccurrence & PMI2  Model based on users searchs

## Listing 6.5

In [4]:
#Calculation:
spark.sql('''
select k1.keyword as keyword1, k2.keyword as keyword2, count(distinct k1.user) users_cooc
from user_searches k1 join user_searches k2 on k1.user = k2.user where k1.keyword > k2.keyword 
group by k1.keyword, k2.keyword ''').createOrReplaceTempView('keywords_users_cooc')

spark.sql('''
select keyword ,  count(distinct user) users_occ
from user_searches 
group by keyword ''').createOrReplaceTempView('keywords_users_oc')


In [5]:
#Show Results
spark.sql('''select * from keywords_users_oc order by users_occ desc''').show(10)
spark.sql('''select count(1) as keywords_users_cooc from keywords_users_cooc''').show()
spark.sql('''select * from keywords_users_cooc order by users_cooc desc''').show(10)

+-----------+---------+
|    keyword|users_occ|
+-----------+---------+
|     lcd tv|     8449|
|       ipad|     7749|
|hp touchpad|     7144|
|  iphone 4s|     4642|
|   touchpad|     4019|
|     laptop|     3625|
|    laptops|     3435|
|      beats|     3282|
|       ipod|     3164|
| ipod touch|     2992|
+-----------+---------+
only showing top 10 rows

+-------------------+
|keywords_users_cooc|
+-------------------+
|             244876|
+-------------------+

+-------------+---------------+----------+
|     keyword1|       keyword2|users_cooc|
+-------------+---------------+----------+
|green lantern|captain america|        23|
|    iphone 4s|         iphone|        21|
|       laptop|      hp laptop|        20|
|         thor|captain america|        18|
|         bose|          beats|        17|
|   skullcandy|          beats|        17|
|    iphone 4s|       iphone 4|        17|
|      laptops|         laptop|        16|
|      macbook|            mac|        16|
|         t

## Listing 6.6

In [6]:
#Calculation:
spark.sql('''
select k1.keyword as k1, k2.keyword as k2, k1_k2.users_cooc, k1.users_occ as n_users1,k2.users_occ as n_users2,
log(pow(k1_k2.users_cooc,2) / (k1.users_occ*k2.users_occ)) as pmi2
from keywords_users_cooc as k1_k2 
join
keywords_users_oc as k1 on k1_k2.keyword1= k1.keyword
join
keywords_users_oc as k2 on k1_k2.keyword2 = k2.keyword
''').createOrReplaceTempView('user_related_keywords_pmi')


In [7]:
#Show Results:
spark.sql( '''
select * from user_related_keywords_pmi where users_cooc >5 order by pmi2 desc
''').show(10)

+-----------------+--------------------+----------+--------+--------+------------------+
|               k1|                  k2|users_cooc|n_users1|n_users2|              pmi2|
+-----------------+--------------------+----------+--------+--------+------------------+
|  iphone 4s cases|      iphone 4 cases|        10|     158|     740|-7.064075033237091|
|     sony laptops|          hp laptops|         8|     209|     432|-7.251876756849249|
|otterbox iphone 4|            otterbox|         7|     122|     787|-7.580428995040033|
|    green lantern|     captain america|        23|     963|    1091|-7.593914965772897|
|          kenwood|              alpine|        13|     584|     717|-7.815078108504774|
|      sony laptop|         dell laptop|        10|     620|     451|-7.936016631553724|
|   wireless mouse|           godfather|         6|     407|     248|-7.938722993151467|
|       hp laptops|        dell laptops|         6|     432|     269| -8.07961802938984|
|      mp3 players|  

## Listing 6.7

In [8]:
#Calculation:
spark.sql('''
select  *, (r1 + r2 /( r1 * r2))/2 as comp_score from (
 select *, 
   rank() over (partition by 1 order by users_cooc desc )  r1 , 
   rank() over (partition by 1 order by pmi2 desc )  r2  
  from user_related_keywords_pmi ) a  '''
).createOrReplaceTempView('users_related_keywords_comp_score')

In [9]:
#Show Results:
spark.sql( '''
  select k1, k2, users_cooc, pmi2, r1, r2, comp_score 
  from users_related_keywords_comp_score
  order by comp_score asc
''').show(20)


+-------------+---------------+----------+-------------------+---+------+------------------+
|           k1|             k2|users_cooc|               pmi2| r1|    r2|        comp_score|
+-------------+---------------+----------+-------------------+---+------+------------------+
|green lantern|captain america|        23| -7.593914965772897|  1|  8626|               1.0|
|    iphone 4s|         iphone|        21|-10.216737746029027|  2| 56156|              1.25|
|       laptop|      hp laptop|        20| -9.132682838345458|  3| 20383|1.6666666666666667|
|         thor|captain america|        18| -8.483026598234463|  4| 13190|             2.125|
|         bose|          beats|        17|-10.074222345094169|  5| 51916|               2.6|
|    iphone 4s|       iphone 4|        17| -10.07559536143275|  5| 51964|               2.6|
|   skullcandy|          beats|        17|  -9.00066454587719|  5| 18792|               2.6|
|         thor|  green lantern|        16| -8.593796095512285|  8| 140

###  Create Cooccurrence & PMI2  Model based on product interaction

## Listing 6.8

In [10]:
#Calculation:
spark.sql("""
  select lower(searches.target) as keyword, searches.user as user, clicks.target as product 
  from signals as searches right join signals as clicks on searches.query_id = clicks.query_id 
  where searches.type='query' and clicks.type = 'click'
""").createOrReplaceTempView('keyword_click_product')


In [11]:
#Show Results:
print("Original signals format: ")
spark.sql(''' select * from signals where type='query' ''').show(3, truncate=False)
print("Simplified signals format: ")
spark.sql(''' select * from keyword_click_product ''').show(3)

Original signals format: 
+------------------------------------+-----------+-----------------------+-------------+-----+-------+
|id                                  |query_id   |signal_time            |target       |type |user   |
+------------------------------------+-----------+-----------------------+-------------+-----+-------+
|00010a48-38c0-4443-a256-1064f88cef0d|u304570_0_1|2019-09-07 09:32:45.142|power adapter|query|u304570|
|00027334-237a-45b0-92d9-e629f17ffa01|u26003_0_1 |2020-04-10 05:55:59.609|jump drive   |query|u26003 |
|000309a8-c9c1-49d6-99d4-cf756360681e|u707571_0_1|2019-11-20 10:36:55.464|Thor blu ray |query|u707571|
+------------------------------------+-----------+-----------------------+-------------+-----+-------+
only showing top 3 rows

Simplified signals format: 
+----------------+-------+------------+
|         keyword|   user|     product|
+----------------+-------+------------+
|     dc universe|u100011|883929194629|
|leandria johnson|u100013|879645002218|


## Listing 6.9

In [12]:
#Calculation:
spark.sql("""
select k1.keyword as k1, k2.keyword as k2, sum(p1) n_users1,sum(p2) n_users2, 
sum(p1+p2) as users_cooc, count(1) n_products
from
(select keyword, product, count(1) as p1 from keyword_click_product group by keyword, product) as k1 
join
(select keyword, product, count(1) as p2 from keyword_click_product group by keyword, product) as k2
on k1.product = k2.product
where k1.keyword > k2.keyword 
group by k1.keyword, k2.keyword
""").createOrReplaceTempView('keyword_click_product_cooc')

In [13]:
#Show Results:
spark.sql('''select count(1) as keyword_click_product_cooc from keyword_click_product_cooc''').show()
spark.sql('''select * from keyword_click_product_cooc order by n_products desc''').show(20)

+--------------------------+
|keyword_click_product_cooc|
+--------------------------+
|                   1579710|
+--------------------------+

+--------------+-------------+--------+--------+----------+----------+
|            k1|           k2|n_users1|n_users2|users_cooc|n_products|
+--------------+-------------+--------+--------+----------+----------+
|       laptops|       laptop|    3251|    3345|      6596|       187|
|       tablets|       tablet|    1510|    1629|      3139|       155|
|        tablet|         ipad|    1468|    7067|      8535|       146|
|       tablets|         ipad|    1359|    7048|      8407|       132|
|       cameras|       camera|     637|     688|      1325|       116|
|          ipad|        apple|    6706|    1129|      7835|       111|
|      iphone 4|       iphone|    1313|    1754|      3067|       108|
|    headphones|  head phones|    1829|     492|      2321|       106|
|     computers|     computer|     536|     392|       928|        98|
| 

## Listing 6.10

In [14]:
#Calculation:
spark.sql("""
select keyword, count(1) as n_users from keyword_click_product group by keyword 
""").createOrReplaceTempView('keyword_click_product_oc')

In [15]:
#Show Results:
spark.sql('''select count(1) as keyword_click_product_oc from keyword_click_product_oc''').show()
spark.sql('''select * from keyword_click_product_oc order by n_users desc''').show(20)

+------------------------+
|keyword_click_product_oc|
+------------------------+
|                   13744|
+------------------------+

+------------+-------+
|     keyword|n_users|
+------------+-------+
|        ipad|   7554|
| hp touchpad|   4829|
|      lcd tv|   4606|
|   iphone 4s|   4585|
|      laptop|   3554|
|       beats|   3498|
|     laptops|   3369|
|        ipod|   2949|
|  ipod touch|   2931|
|      ipad 2|   2842|
|      kindle|   2833|
|    touchpad|   2785|
|   star wars|   2564|
|      iphone|   2430|
|beats by dre|   2328|
|     macbook|   2313|
|  headphones|   2270|
|        bose|   2071|
|         ps3|   2041|
|         mac|   1851|
+------------+-------+
only showing top 20 rows



## Listing 6.11

In [16]:
# calculate PMI2, per Listing 6.6

#Calculation:
spark.sql('''
select k1.keyword as k1, k2.keyword as k2, k1_k2.users_cooc, k1.n_users as n_users1,k2.n_users as n_users2,
log(pow(k1_k2.users_cooc,2)/(k1.n_users*k2.n_users)) as pmi2
from
keyword_click_product_cooc as k1_k2 
join
keyword_click_product_oc as k1 on k1_k2.k1 = k1.keyword
join
keyword_click_product_oc as k2 on k1_k2.k2 = k2.keyword
''').createOrReplaceTempView('product_related_keywords_pmi')

In [17]:
#Show Results:
spark.sql('''select count(1) as related_keywords_pmi from product_related_keywords_pmi''').show()
spark.sql('''select * from product_related_keywords_pmi order by pmi2 desc''').show(20)

+--------------------+
|related_keywords_pmi|
+--------------------+
|             1579710|
+--------------------+

+-------------------+-------------------+----------+--------+--------+------------------+
|                 k1|                 k2|users_cooc|n_users1|n_users2|              pmi2|
+-------------------+-------------------+----------+--------+--------+------------------+
|          pad pivot|        hp touchpad|      4022|       1|    4829| 8.116674454791653|
|     hp touchpad 32|        hp touchpad|      4022|       1|    4829| 8.116674454791653|
|        hp touchpad|    hp tablet 32 gb|      4022|    4829|       1| 8.116674454791653|
|        hp touchpad|     hp tablet 32gb|      4022|    4829|       1| 8.116674454791653|
|           touchpad|     hp touchpad 32|      2350|    2785|       1| 7.592338061915025|
|           touchpad|    hp tablet 32 gb|      2350|    2785|       1| 7.592338061915025|
|           touchpad|     hp tablet 32gb|      2350|    2785|       1| 7.5

In [18]:
# calculate comp_score, per Listing 6.7

#Calculation:
spark.sql('''
select  *, (r1 + r2 /( r1 * r2))/2 as comp_score from (
 select *, 
   rank() over (partition by 1 order by users_cooc desc )  r1 , 
   rank() over (partition by 1 order by pmi2 desc )  r2  
  from product_related_keywords_pmi ) a  '''
).createOrReplaceTempView('product_related_keywords_comp_score')

In [19]:
#Show Results:
spark.sql( '''
  select count(1) product_related_keywords_comp_scores from product_related_keywords_comp_score
''').show()

spark.sql( '''
  select k1, k2, n_users1, n_users2, pmi2, comp_score 
  from product_related_keywords_comp_score
  order by comp_score asc
''').show(20)

+------------------------------------+
|product_related_keywords_comp_scores|
+------------------------------------+
|                             1579710|
+------------------------------------+

+----------+-----------+--------+--------+------------------+------------------+
|        k1|         k2|n_users1|n_users2|              pmi2|        comp_score|
+----------+-----------+--------+--------+------------------+------------------+
|      ipad|hp touchpad|    7554|    4829|1.2318940540272372|               1.0|
|    ipad 2|       ipad|    2842|    7554| 1.430517155037946|              1.25|
|    tablet|       ipad|    1818|    7554|1.6685364924472557|1.6666666666666667|
|  touchpad|       ipad|    2785|    7554|1.2231908670315748|             2.125|
|   tablets|       ipad|    1627|    7554|1.7493143317791537|               2.6|
|     ipad2|       ipad|    1254|    7554|1.9027023623302282|3.0833333333333335|
|      ipad|      apple|    7554|    1814|1.4995901756327583|3.571428571428

Up next: [Misspelling detection and correction](../ch06/3.spell-correction.ipynb)